<a href="https://colab.research.google.com/github/pranalibose/LLM_Workshop/blob/main/LLM_Workshop_Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install transformers datasets evaluate rouge_score accelerate --use-deprecated=legacy-resolver

In [2]:
import transformers

transformers.__version__

'4.52.4'

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
from huggingface_hub import list_datasets

datasets_list = list_datasets()

num = 0
for dataset in datasets_list:
    print(dataset)
    num += 1
    if num == 5:
        break

DatasetInfo(id='nvidia/Nemotron-Personas', author='nvidia', sha='65887f26ae478d7d2df68438b6c10d58d037b76d', created_at=datetime.datetime(2025, 6, 9, 4, 12, 11, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2025, 6, 9, 18, 21, 17, tzinfo=datetime.timezone.utc), private=False, gated=False, disabled=False, downloads=7486, downloads_all_time=None, likes=96, paperswithcode_id=None, tags=['task_categories:text-generation', 'language:en', 'license:cc-by-4.0', 'size_categories:100K<n<1M', 'format:parquet', 'modality:text', 'library:datasets', 'library:pandas', 'library:mlcroissant', 'library:polars', 'region:us', 'synthetic', 'personas', 'NVIDIA'], trending_score=96, card_data=None, siblings=None, xet_enabled=None)
DatasetInfo(id='fka/awesome-chatgpt-prompts', author='fka', sha='68ba7694e23014788dcc8ab5afe613824f45a05c', created_at=datetime.datetime(2022, 12, 13, 23, 47, 45, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2025, 1, 6, 0, 2, 53, tzinfo=datetime.ti

In [7]:
#!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from datasets import load_dataset

cnn_news_summary_ds = load_dataset("cnn_dailymail", "3.0.0")

cnn_news_summary_ds

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

In [ ]:
cnn_news_summary_ds = load_dataset("cnn_dailymail", "3.0.0", split = 'train[:1%]')

cnn_news_summary_ds

Dataset({
    features: ['article', 'highlights', 'id'],
    num_rows: 2871
})

In [ ]:
cnn_news_summary_ds.shape

(2871, 3)

In [ ]:
cnn_news_summary_ds.features

{'article': Value(dtype='string', id=None),
 'highlights': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None)}

In [ ]:
cnn_news_summary_ds = cnn_news_summary_ds.train_test_split(test_size = 0.2, seed=42)

cnn_news_summary_ds

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 2296
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 575
    })
})

In [ ]:
def clean_text(example):
    for txt in ['article', 'highlights']:
        example[txt] = example[txt].replace('\n', '')
        example[txt] = example[txt].replace('--', '')
        example[txt] = example[txt].replace('\\', '')
        example[txt] = example[txt].replace('/', '')
        example[txt] = example[txt].replace('"', '')
        example[txt] = example[txt].replace('``', '')
        return example

In [ ]:
cleaned_cnn_news_summary_ds = cnn_news_summary_ds.map(clean_text)

cleaned_cnn_news_summary_ds

Map:   0%|          | 0/2296 [00:00<?, ? examples/s]

Map:   0%|          | 0/575 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 2296
    })
    test: Dataset({
        features: ['article', 'highlights', 'id'],
        num_rows: 575
    })
})

# Using Pre-trained model to extract summaries

In [ ]:
TEXT_INDEX = 33

example_text = cleaned_cnn_news_summary_ds['test']['article'][TEXT_INDEX]

example_text

"BARCELONA, Spain (CNN)  Cell phone makers Tuesday pledged to end one of modern life's chief frustrations - and introduce a universal charger for handsets by 2012. An estimated 1.2 billion cell phones were sold in 2008, at least half of which were replacement handsets. The GSMA (Groupe Speciale Mobile Association), which represents more than 750 of the world's cell phone operators, made the announcement at its annual Mobile World Congress in Barcelona Tuesday. Under the scheme, phone makers have pledged that a majority of new handset models will include the universal charger by January 1 2012. The planned device will use a micro USB plug. Aside from bringing relief to drawers stuffed full of redundant chargers, the GSMA stressed that the new device would reduce raw materials. The mobile industry has a pivotal role to play in tackling environmental issues and this programme is an important step that could lead to huge savings in resources, not to mention convenience for consumers, said 

In [ ]:
MODEL_NAME = 't5-small'

from transformers import pipeline

prefix = 'summarize: '

summarizer = pipeline(task='summarization', model=MODEL_NAME)

summary_text = summarizer(prefix + example_text)

summary_text

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

[{'summary_text': '1.2 billion cell phones were sold in 2008, at least half of them replacement handsets . a majority of new handset models will include the universal charger by 2012 . the GSMA said the new device would reduce raw materials .'}]

In [ ]:
summarizer = pipeline(task='summarization', model=MODEL_NAME, truncation=True)

summary_text = summarizer(prefix + example_text)

summary_text

[{'summary_text': '1.2 billion cell phones were sold in 2008, at least half of them replacement handsets . a majority of new handset models will include the universal charger by 2012 . the GSMA said the new device would reduce raw materials .'}]

In [ ]:
ref_text = cleaned_cnn_news_summary_ds['test']['highlights'][TEXT_INDEX]

ref_text

"The GSMA represents more than 750 of the world's cell-phone operators .\nPlan is that majority of new models will have micro-USB charger by January 2012 .\nAn estimated 1.2 billion cell phones sold in 2008, at least half replacement handsets ."

# Evaluation of summarization using ROUGE

In [ ]:
import evaluate

rouge = evaluate.load('rouge')

rouge

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

In [ ]:
rouge_score = rouge.compute(predictions=[summary_text[0]['summary_text']], references=[ref_text], use_stemmer=True)

rouge_score

{'rouge1': 0.6097560975609757,
 'rouge2': 0.37500000000000006,
 'rougeL': 0.31707317073170727,
 'rougeLsum': 0.5853658536585366}

# Generate summaries for multiple articles in the dataset

In [ ]:
article_texts = cleaned_cnn_news_summary_ds['test']['article']

article_summaries = cleaned_cnn_news_summary_ds['test']['highlights']

In [ ]:
# Execution time ~ 1 min

from tqdm import tqdm

candidate_summaries = []

prefix = 'summarize: '

for i, text in enumerate(tqdm(article_texts[:10])):
    candidate = summarizer(prefix + text)
    candidate_summaries.append(candidate[0]['summary_text'])

100%|██████████| 10/10 [00:46<00:00,  4.68s/it]


In [ ]:
result_agg = rouge.compute(predictions=candidate_summaries, references=article_summaries[:10], use_stemmer=True)

result_agg

{'rouge1': 0.2642947501054136,
 'rouge2': 0.08402423995414929,
 'rougeL': 0.18056360139747685,
 'rougeLsum': 0.22666311106098302}

In [ ]:
result_unagg = rouge.compute(predictions=candidate_summaries, references=article_summaries[:10], use_stemmer=True, use_aggregator=False)

result_unagg

{'rouge1': [0.25974025974025977,
  0.3260869565217391,
  0.1686746987951807,
  0.3555555555555555,
  0.3125,
  0.27956989247311825,
  0.21333333333333335,
  0.3287671232876712,
  0.2823529411764706,
  0.09999999999999999],
 'rouge2': [0.10666666666666667,
  0.06666666666666665,
  0.0,
  0.15909090909090912,
  0.10638297872340426,
  0.10989010989010989,
  0.08219178082191782,
  0.11267605633802817,
  0.09638554216867469,
  0.0],
 'rougeL': [0.23376623376623376,
  0.19565217391304346,
  0.1686746987951807,
  0.17777777777777776,
  0.18749999999999997,
  0.2150537634408602,
  0.10666666666666667,
  0.3013698630136986,
  0.1176470588235294,
  0.09999999999999999],
 'rougeLsum': [0.23376623376623376,
  0.26086956521739124,
  0.1686746987951807,
  0.2888888888888889,
  0.2708333333333333,
  0.23655913978494622,
  0.18666666666666668,
  0.3287671232876712,
  0.18823529411764706,
  0.09999999999999999]}

In [ ]:
import numpy as np

result_unagg_rsum = np.array(result_unagg['rouge2'])

np.argmax(result_unagg_rsum), np.argmin(result_unagg_rsum)

(3, 2)

In [ ]:
import pandas as pd

act_vs_pred_summaries_df = pd.DataFrame(list(zip(candidate_summaries, article_summaries[:10])),
                                        columns = ['Predicted_summaries', 'Reference_summaries'])

act_vs_pred_summaries_df.head()

,Predicted_summaries,Reference_summaries
0,a new exhibit at the skirball cultural center ...,"Superman, Batman, others born during Great Dep..."
1,"GEROLSTEIN, Germany Mineral water company gero...",The Gerolsteiner company is ending its sponsor...
2,penguins are warm-weather birds that breed in ...,Activists rescue group of young penguins who m...
3,sheriff says he will not face criminal charges...,Michael Phelps won't be charged in connection ...
4,state officials replaced a $40-a-week child su...,Virginia is one of two dozen states that use d...


In [ ]:
print('Actual Summary')
print(act_vs_pred_summaries_df._get_value(3, 'Predicted_summaries'))
print()
print('Predicted Summary')
print(act_vs_pred_summaries_df._get_value(3, 'Reference_summaries'))

Actual Summary
sheriff says he will not face criminal charges in connection with the incident . he admitted regrettable behavior after a photo of him using a bong was published . the 23-year-old won a record eight gold medals at the 2008 olympics .

Predicted Summary
Michael Phelps won't be charged in connection with bong picture, sheriff says .
Sheriff: "We didn't have enough where we could go arrest him"
British newspaper published picture of Olympic champion using a bong .
Sheriff defends probe, saying he would have been criticized if he hadn't investigated .


In [ ]:
print('Actual Summary')
print(act_vs_pred_summaries_df._get_value(2, 'Predicted_summaries'))
print()
print('Predicted Summary')
print(act_vs_pred_summaries_df._get_value(2, 'Reference_summaries'))

Actual Summary
penguins are warm-weather birds that breed in large colonies in southern Argentina and Chile . they migrate north between March and September, following their favorite fish, the anchovy . the birds were stranded and emaciated .

Predicted Summary
Activists rescue group of young penguins who migrated too far north on Brazil coast .
Penguins loaded onto a C-130 Hercules military plane and flown south .
To cheers of onlookers, penguins return to ocean near their feeding grounds .
Changing currents, temperatures may have led the confused birds off course .


# Fine Tuning T5 Model

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
prefix = 'summarize: '

def preprocess_function(examples):

    inputs = [prefix + doc for doc in examples['article']]
    model_inputs = tokenizer(inputs, max_length = 1024, truncation = True)

    labels = tokenizer(text_target = examples['highlights'], max_length = 128, truncation = True)

    model_inputs['labels'] = labels['input_ids']

    return model_inputs

In [ ]:
tokenized_cnn_news_summary_ds = cleaned_cnn_news_summary_ds.map(preprocess_function, batched = True)

tokenized_cnn_news_summary_ds

Map:   0%|          | 0/2296 [00:00<?, ? examples/s]

Map:   0%|          | 0/575 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2296
    })
    test: Dataset({
        features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 575
    })
})

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = MODEL_NAME)

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens = True)

    result = rouge.compute(predictions = decoded_preds, references = decoded_labels, use_stemmer = True)

    prediction_length = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result['generated_length'] = np.mean(prediction_length)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir = 'cnn_news_summary_model_trained_on_reduced_data',
    eval_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    weight_decay = 0.01,
    save_total_limit = 3,
    num_train_epochs = 1,
    predict_with_generate = True,
    fp16 = True,
    push_to_hub = True
)

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_cnn_news_summary_ds['train'],
    eval_dataset = tokenized_cnn_news_summary_ds['test'],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Generated Length
1,No log,1.831446,0.234000,0.097100,0.191700,0.191800,18.991300


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=144, training_loss=2.0721719529893665, metrics={'train_runtime': 161.776, 'train_samples_per_second': 14.192, 'train_steps_per_second': 0.89, 'total_flos': 621489552359424.0, 'train_loss': 2.0721719529893665, 'epoch': 1.0})

In [ ]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

events.out.tfevents.1718342621.9f1cbee3c31e.274.0:   0%|          | 0.00/6.75k [00:00<?, ?B/s]

Upload 7 LFS files:   0%|          | 0/7 [00:00<?, ?it/s]

events.out.tfevents.1718342706.9f1cbee3c31e.274.1:   0%|          | 0.00/6.75k [00:00<?, ?B/s]

events.out.tfevents.1718342859.9f1cbee3c31e.274.2:   0%|          | 0.00/6.76k [00:00<?, ?B/s]

events.out.tfevents.1718343475.9f1cbee3c31e.274.3:   0%|          | 0.00/6.76k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pranalibose/cnn_news_summary_model_trained_on_reduced_data/commit/a22a24602d0356ff03cd17ebfe082b8159400303', commit_message='End of training', commit_description='', oid='a22a24602d0356ff03cd17ebfe082b8159400303', pr_url=None, pr_revision=None, pr_num=None)